## Motivation und Ziel
Durch die anhaltende Teuerung auf dem Wohnungsmarkt und nicht zuletzt verstärkt durch die steigenden Lebensmittelpreise sind die Lebenshaltungskosten in den letzten Jahren deutlich gestiegen. Die Preissteigerung im Wohnungsmarkt ist dabei ein bereits länger anhaltender Trend. Zwar handelt es sich um eine allgemeine Entwicklung, jedoch bestehen große regionale Unterschiede hinsichtlich der Intensität der Teuerung. Dies weckte das Interesse daran, die zugrunde liegenden Mechanismen besser zu verstehen. Speziell, welche regionalen Unterschiede gibt es und welche Faktoren bei der Preisbildung eine Rolle spielen.
Im Rahmen dieses Seminars bot sich die Gelegenheit, die Einflussfaktoren auf Wohnungspreise aus der Perspektive des maschinellen Lernens zu untersuchen. Daraus ergab sich die Idee und Grundlage für unsere spätere Forschungsfrage: Die Bestandsmiete mithilfe von Sozioökonomischen zu lernen und das resultierende Vorhersagemodell zu analysieren, um die wichtigsten Parameter zu ermitteln.

## Vorgehen
Aufgrund des begrenzten zeitlichen Rahmens und der Natur unserer Fragestellung haben wir uns für einen Ad-hoc-Prozess entschieden, bei dem wir unsere genaue Forschungsfrage iterativ an die verfügbaren Daten angepasst haben. Um unsere Grundidee umzusetzen, haben wir uns nach ausführlicher Recherche für die Daten aus dem *Deutschlandatlas* entschieden.

Der *Deutschlandatlas* ist ein gemeinsames Projekt mehrerer Bundesministerien und stellt die Lebensverhältnisse in Deutschland in 84 Karten dar.  
Der Datensatz ist aufgrund seiner Herkunft sehr umfangreich und qualitativ hochwertig.
Er enthält nicht nur die für unser Projektvorhaben relevanten sozioökonomischen Daten, sondern auch unsere Zielvariable: die Bestandsmiete.


Die Daten für die Karten lagen in verschiedenen Dateien vor und wiesen unterschiedliche Granularität auf. Einige Datensätze waren nur auf Ebene der Gebietskörperschaften oder Verbandsgemeinden verfügbar, während die meisten auf Gemeindeebene erfasst wurden.
Um diese Daten zusammenzuführen, haben wir mithilfe von Excel Power Query alle Tabellen anhand des Gemeindeschlüssels in eine gemeinsame Tabelle überführt.  
Dabei entstand eine Gesamttabelle mit rund 10.000 Gemeinden und den zugehörigen 84 Merkmalen (Features).
Als nächsten Schritt haben wir die Daten bereinigt. Zunächst wurden Features entfernt,
1. die zu viele fehlende Werte aufwiesen, oder
2. die zu ähnlich zur Zielvariable waren (z. B. die Baulandpreise).
Anschließend haben wir Datenpunkte (Gemeinden) ausgeschlossen, die selbst zu viele fehlende Werte hatten. Dies hatte meist zwei Ursachen:  
Entweder handelte es sich um Gemeinden ohne Einwohner oder um kürzlich neu entstandene Gemeinden infolge kommunaler Reformen. 
Vereinzelte fehlende Werte haben wir manuell mit dem Mittelwert der benachbarten Werte aufgefüllt.  Dies war möglich, da die Daten durch den Gemeindeschlüssel lokal sortiert vorlagen.

Dadurch erhielten wir den unseren verwendbaren kompletten Datensatz mit nun 78 Feature und immer noch rund 10.000 Datenpunkten hiermit konnten wir unsere finale konkretisieren Forschungsfrage: Welchen Einfluss haben gezielt ausgewählte sozioökonomische Merkmale auf die Bestandsmiete pro Quadratmeter auf Gemeindeebene, wie gut lassen sich diese mithilfe von maschinellem Lernen vorhersagen, und inwieweit ist die Modellinterpretation nachvollziehbar? 

### Exkurs: OpenStreetMap (OSM)
Unser Ziel war es, zusätzliche Merkmale (Features) zu identifizieren, da unsere Forschungsfrage sehr explorativen Charakter hatte und wir möglichst viele Informationen einbeziehen wollten.  Daher extrahierten wir ergänzende Daten aus *OpenStreetMap (OSM).*
Diese Daten wurden ebenfalls bereinigt, und wir konnten rund 30 zusätzliche Merkmale generieren.  Allerdings zeigten Tests, dass diese Merkmale durchweg nur eine sehr geringe Feature-Importance aufwiesen. Wir vermuten, dass dies an der im Vergleich zu den staatlich erhobenen Daten niedrigeren Qualität der *OSM*-Daten liegt.  Aus diesem Grund entschieden wir uns, diese Datenquelle im finalen Projekt nicht weiter zu verwenden und arbeiteten ausschließlich mit den Daten aus dem *Deutschlandatlas.*

## Auswahl des Modell
Da wir relativ ehrgeizig an das Projekt herangegangen sind, haben wir weiterhin ad-hoc gearbeitet. Nach ersten Recherchen standen uns mehrere Modelle zur Prognose zur Auswahl.  
Dabei stellten sich insbesondere *Random Forest* und *XGBoost* als vielversprechend heraus – laut empirischer Forschung erzielen diese Modelle bei tabellarischen Datensätzen häufig die besten Ergebnisse.

Trotzdem wollten wir auch ein *Deep-Learning-Modell* trainieren. Dafür gab es zwei Gründe:  
Erstens wollten wir gezielt ein sogenanntes *Black-Box-Modell* einsetzen, um die im Seminar behandelten *post-hoc, lokal modellagnostischen* Interpretationsmethoden wie *LIME* und *SHAP* anwenden zu können, die typischerweise für schwer interpretierbare Modelle entwickelt wurden.

Wir führten erste Tests mit *FT-Transformer* und *TabNet* durch, die wir zwar grundsätzlich als vielversprechende Ansätze einschätzten. Aufgrund unserer begrenzten Erfahrung mit Deep Learning, des hohen Zeitaufwands sowie der erhöhten Komplexität dieser Modelle verwarfen wir diese Optionen jedoch wieder.

Stattdessen entschieden wir uns aus *didaktischen Gründen* für ein Multilayer Perceptron *(MLP)*, das wir im Modul _Einführung in die Künstliche Intelligenz_ kennengelernt hatten, welches parallel zu diesem Seminar stattfand. Das _MLP_ war für uns besser verständlich und praktikabler umsetzbar.

Das Modell haben wir mithilfe von *PyTorch* umgesetzt, um die privaten Ressourcen einer leistungsstarken GPU nutzen zu können. Dies ermöglichte durch Parallelisierung ein deutlich beschleunigtes Training.

## Feature Engineering 
Da wir durch unsere verschiedenen Experimente bereits gut mit den Daten vertraut waren und bereits SHAP-Summary-Plots erstellt hatten, hatten wir ein gutes Gespür dafür, welche Merkmale einen starken Einfluss auf das Modell hatten – und welche nicht.

Dabei stellten wir fest, dass der *Wohnungsleerstand* in unseren Testmodellen eine sehr hohe SHAP-Feature-Importance aufwies und somit einen übermäßig starken Einfluss auf den Mietpreis hatte. Aus diesem Grund entschieden wir uns, dieses Merkmal aus dem Modell zu entfernen.
Im Rahmen der begleitenden Übung wurden wir zusätzlich darauf hingewiesen, dass es insbesondere beim Trainieren eines MLP sinnvoll ist, *stark korrelierende Merkmale* zu eliminieren, um die Modellqualität zu verbessern.

Beispielsweise zeigte sich eine nahezu perfekte Korrelation (r ≈ 0,9) zwischen den Variablen *Arbeitslosenquote* und *Soziale Mindestsicherung*. Ebenso gab es starke Überschneidungen zwischen *Teilzeitquote männlich* und *Teilzeitquote weiblich*. In solchen Fällen entschieden wir uns jeweils dafür, stattdessen eine aggregierte Variable wie die *Teilzeitquote insgesamt* zu verwenden bzw. stark redundante Merkmale vollständig zu entfernen. Durch unsere Korrelationsanalyse sowie unseren initialen Verdacht konnten wir zudem feststellen, dass der *Wohnungsleerstand* mit vielen anderen Variablen *stark korreliert*.  
Dies bestätigte unsere Entscheidung, dieses Merkmal aus dem Modell zu entfernen.

Auf dieser Basis erstellten wir eine finale *Design-Matrix mit  ? Mermalen und ?Datenpunkten, die wir anschließend *standardisiert* haben, um sie für das Training des MLP-Modells zu verwenden.

## Learning and Limitiations
Aufgrund unserer begrenzten Vorkenntnisse war unser Vorgehen insgesamt eher unstrukturiert. Wir haben jedoch viele verschiedene Modellansätze ausprobiert, was uns wertvolle Einblicke, aber auch zahlreiche *Limitationen* aufgezeigt hat.

Besonders deutlich wurde dabei, *wie wichtig durchdachtes Feature Engineering ist* – nicht nur, weil es maßgeblich die Modellgüte beeinflusst, sondern auch, weil es sehr zeitaufwendig ist. Jeder neue Versuch erforderte ein erneutes Training der Modelle, was insbesondere bei komplexeren Ansätzen viel Zeit in Anspruch nahm.  
Wir haben dabei gelernt, dass es sinnvoll ist, sich frühzeitig an bewährten Best-Practice-Ansätzen zu orientieren, um unnötige Rückschritte zu vermeiden.

Zudem führte die nicht-deterministische Natur von *Deep-learning* Modellen zu stark schwankenden Ergebnissen selbst bei identischer Datenbasis. Auch wenn viele dieser Resultate zunächst plausibel wirkten, wurde uns bewusst, dass deren Interpretation häufig von einem gewissen Confirmation Bias* beeinflusst war.